In [ ]:
import glob
from functools import cache

import duckdb
import folium
import matplotlib as plt
import pandas as pd

In [ ]:
table_name_counts = "road_traffic_counts"
table_name_stations = "station_reference"


csv_files_path = "../data/road_traffic_counts_hourly_permanent/*.csv"
station_reference_csv = "../data/road_traffic_counts_station_reference.csv"

In [ ]:
csv_files = glob.glob(csv_files_path)

In [ ]:
@cache
def load_data_duckdb():
    con = duckdb.connect()

    con.execute(
        f"CREATE TABLE {table_name_stations} AS SELECT * FROM read_csv_auto('{station_reference_csv}')"
    )

    # Assume the first file defines the table structure
    con.execute(f"CREATE TABLE {table_name_counts} AS SELECT * FROM read_csv_auto('{csv_files[0]}')")

    # For each subsequent file, insert the data into the existing table
    for csv_file in csv_files[1:]:
        con.execute(f"INSERT INTO {table_name_counts} SELECT * FROM read_csv_auto('{csv_file}')")

    df = con.sql(f"SELECT * FROM {table_name_counts}").to_df()
    stats = pd.DataFrame(df.describe())
    stats_all = pd.DataFrame(df.describe(include="all"))

    # stats_objects = df.describe(include=[object])
    return df, stats, stats_all, con

In [ ]:
df, stats, stats_all, con = load_data_duckdb()

In [ ]:
def show_schema(table_name):
    schema = con.execute(f"DESCRIBE {table_name}").fetch_df()
    return schema[["column_name", "column_type", "null"]]

In [ ]:
show_schema(table_name_counts)

In [ ]:
show_schema(table_name_stations)

In [ ]:
def df_station_id():
    exclude_station_id = ["18031", "11139", "19035"]
    exclude_station_id_sql = ", ".join(f"'{id}'" for id in exclude_station_id)

    station_sql_query = f"""
        SELECT * FROM {table_name_stations}
        WHERE full_name ILIKE '%Victoria Road%'
        AND station_id NOT IN ({exclude_station_id_sql});
    """

    station_df = con.sql(station_sql_query).to_df()
    return station_df

In [ ]:
station_df = df_station_id()
vic_rd_stations = station_df["station_key"].unique().tolist()

In [ ]:
result = con.execute(f"SELECT COUNT(*) FROM {table_name_counts}").fetchone()

# result now contains the count of rows, which is the first (and only) element in the returned tuple
row_count = result[0]

print(f"Number of rows in {table_name_counts}: {row_count}")
assert len(df) == row_count

In [ ]:
# Convert year_start to datetime
year_start_datetime = pd.to_datetime("2018-01-01")

# Get the current date as datetime
current_date_datetime = pd.to_datetime("now")

In [ ]:
def plot_counts_for_station_key_by_hour(station_key, hour, year_start=year_start_datetime):
    df = con.sql(
        f"SELECT date, hour_{hour:02}, daily_total FROM {table_name_counts} WHERE station_key = {station_key} AND classification_seq = 2 ORDER BY date ASC"
    ).to_df()
    quantile_max = round(df[f"hour_{hour:02}"].quantile(0.999) / 100) * 100  # round to nearest 100
    df.plot(
        x="date", y=f"hour_{hour:02}", xlim=[year_start, current_date_datetime], ylim=[0, quantile_max]
    )
    return df

In [ ]:
df_all = {}
quantiles = {}
for hour in range(0, 24):
    df_all[hour] = plot_counts_for_station_key_by_hour("99990010", hour)

In [ ]:
def plot_counts_for_station_key(station_key):
    # df = con.sql(f"SELECT * FROM {table_name_counts} WHERE station_key = {station_key} ORDER BY date").to_df()
    df = con.sql(
        f"SELECT date, daily_total FROM {table_name_counts} WHERE station_key = {station_key} AND classification_seq = 2 ORDER BY date ASC"
    ).to_df()
    if len(df) > 0:
        df.plot(x="date", y="daily_total", legend=f"Station key: {station_key}")
        return df
    else:
        print(f"Station key: {station_key} has no data")
        return station_key

In [ ]:
df_all = {}
station_key_no_data = []

for station_key in vic_rd_stations:
    result = plot_counts_for_station_key(station_key)
    if isinstance(result, pd.DataFrame):
        df_all[station_key] = result
    else:
        station_key_no_data.append(result)

In [ ]:
station_key_no_data

In [ ]:
len(df_all)

In [ ]:
print([table_name[0] for table_name in con.execute("SHOW TABLES;").fetchall()])

In [ ]:
def map_selected_stations(station_df):
    df = station_df.copy()
    # Assuming df has 'wgs84_latitude', 'wgs84_longitude', 'station_id', 'station_key' and 'full_name' columns

    m = folium.Map()
    fg = folium.FeatureGroup()  # Create a feature group

    # Add markers to the feature group with popups
    for _, row in df.iterrows():
        popup_text = f"Station ID (Key): {row['station_id']} ({row['station_key']})<br>Full Name: {row['full_name']}"
        marker = folium.Marker(
            [row["wgs84_latitude"], row["wgs84_longitude"]],
            popup=folium.Popup(popup_text, max_width=450),
        )
        fg.add_child(marker)
    m.add_child(fg)
    m.fit_bounds(fg.get_bounds())
    return m

In [ ]:
m = map_selected_stations(station_df)

In [ ]:
m